In [ ]:
## 필요한 Library Import
import os
import pandas as pd # R의 dataframe 데이터 타입을 참고하여 만든 것이 Pandas Library
import numpy as np

import matplotlib.pyplot as plt
# import seaborn as sns

# %matplotlib inline

In [ ]:
path = 'PF_data.csv'
df = pd.read_csv(path) # csv 파일을 Dataframe으로 불러와 처리
df

In [ ]:
list(df.columns) # 몇몇 columns들이 있는지 확인

# EDA
- 진단 방법 1에 대해서만 진행 

In [ ]:
info_feature = ['SubjectNo','Pf_date_time','Pf_date','Date_birth']
# Independent Variable
clinical_feature = ['Gender','Age','Height','Weight','BMI','BT','hx_cancer']
blood_serum_feature = ['b_WBC','b_Neutro','b_Lympho','b_mono','b_Eosinophil', 
                       'b_Basophil','b_RBC','b_Hb', 'b_plt', 'b_ESR', 
                       'b_CRP', 'b_Protein', 'b_glucose','b_glucose_poct', 'b_ALP', 
                       'b_TB','b_BUN','b_Calcium','b_Chloride','b_Cholesterol',
                       'b_total_CO2','b_Creatinine','b_GGT','b_AST','b_ALT', 
                       'b_Phosphorus', 'b_K','b_Na','b_Uric_Acid','b_albumin',
                       'b_LDH','b_CEA', ]
pleural_fluid_feature = ['SONO','Pf_Color','pf_pH', 'pf_RBC', 'pf_WBC', # SONO 초음파 사용 여부 
                         'pf_PMN_p', 'pf_Lympho_p', 'pf_other_p',
                         'Pf_Lymph_PMN', 'pf_protein', 'pf_glucose',
                         'pf_chloride', 'pf_LD', 'pf_amylase', 'pf_albumin',
                         'pf_ADA', 'pf_CEA']

# Dependent Variable
target_features = ['final_dx_1', 'final_dx_2'] 

print(f'Clinical: {len(clinical_feature)}\nBlood/Serum: {len(blood_serum_feature)}\nPleural Fluid: {len(pleural_fluid_feature)}')

In [ ]:
label_dict = {0:'Excluded', 1:'Transudative',
         2:'Malignant', 3:'Parapneumonic',
         4:'Tuberculous', 5:'Others'}

### 데이터 타입 체크 
- 범주형 변수(Categorical Data): Object or String
- 수치형 변수(Numerical Data): Int64 or Float64

In [ ]:
# df = df[clinical_feature+blood_serum_feature+pleural_fluid_feature+target_features]
print(df.shape)

In [ ]:
df.info() # 변수들의 자료형 파악

In [ ]:
categorical = ['Gender', 'hx_cancer', 'SONO', 'Pf_Color', 'final_dx_1', 'final_dx_2']
numerical = ['Age','Height','Weight','BMI','BT',
            'pf_pH', 'pf_RBC', 'pf_WBC', 'pf_PMN_p', 'pf_Lympho_p', 'pf_other_p','Pf_Lymph_PMN', 
             'pf_protein', 'pf_glucose','pf_chloride', 'pf_LD', 'pf_amylase', 'pf_albumin','pf_ADA', 'pf_CEA'] + blood_serum_feature

print(f'Categorical Data: {len(categorical)}\nNumerical Data: {len(numerical)}')

In [ ]:
df.describe() # 각종 데이터의 통계량을 요약하여 출력해주는 Method

### Missing Value

In [ ]:
# 결측치 개수 파악
df.isnull().sum()

### Dependent Variable 종속변수 Distribution 

In [ ]:
# Graph를 그리기 위한 Colormap 설정
color_map = ['#064273','#76b6c4','#107dac','#005b96','#6497b1','#b3cde0','#8b9dc3','#428bca','#7289da','#5bc0de','#bae1ff']

In [ ]:
# Target 값들의 갯수를 파악하기 위해 필요한 정보만 Dataframe으로 변환 
df_graph = pd.DataFrame(df['final_dx_1'].value_counts()) 
df_graph.sort_index(inplace=True)
df_graph

In [ ]:
# 진단 1에 대한 target 분포 그래프 
fig, ax = plt.subplots()
for i in df_graph.index:
    value = df_graph.loc[i, 'final_dx_1']
    rect = ax.bar(i, value, color=color_map[i])
    height = rect[0].get_height()
    ax.text(i, height, round(value), ha='center', va='bottom')
    # ax.set_xticks([i])
plt.title('Diagnostic Method 1')
plt.xticks(np.arange(len(df_graph)), list(label_dict.values()), rotation=45)
plt.show()

### Numerical Variable Distribution

In [ ]:
# # 각 Feature별 Distribution Graph
# for col in numerical:
#     sns.histplot(df.loc[df[col].notnull(), col], kde=True, stat='density', linewidth=0.1)
#     plt.show()

### Correlation Map

In [ ]:
corrmat = df.corr() # Correlation 

# plt.figure(figsize=(16,12))
fig, ax = plt.subplots(figsize=(8,8))
heatmap = ax.pcolor(corrmat, cmap=plt.cm.Blues)

# put the major ticks at the middle of each cell
ax.set_xticks(np.arange(0.5, len(corrmat.index), 1) + 0.5, corrmat.index, fontsize=8, rotation=90)
ax.set_yticks(np.arange(0.5, len(corrmat.columns), 1), corrmat.columns, fontsize=8)

# want a more natural, table-like display
ax.invert_yaxis()
ax.xaxis.tick_top()
ax.grid(False)

# turn off the frame
ax.set_frame_on(False)
# cax = divider.append_axes('right', size='5%', pad=0.05)
# fig.colorbar(heatmap)
plt.show()

# Preprocessing
- 1차적 전처리를 통한 데이터 보유

In [ ]:
blood_serum_feature = []
pleural_fluid_feature = []
date_time_feature = []
date_feature = []
for c in df.columns:
    if 'date_time' in c:
        date_time_feature.append(c)
    if ('_date' in c) & ('time' not in c):
        date_feature.append(c)
    if ('b_' in c) & ('_date' not in c):
        blood_serum_feature.append(c)
    if ('pf_' in c) | ('Pf_' in c) & ('_date' not in c):
        pleural_fluid_feature.append(c)
        
print(len(blood_serum_feature), len(pleural_fluid_feature), len(date_time_feature), len(date_feature))

In [ ]:
info_feature = ['SubjectNo','Date_birth']
# Independent Variable
clinical_feature = ['Gender','Age','Height','Weight','BMI','BT','hx_cancer', 'SONO']

# Dependent Variable
target_features = ['final_dx_1', 'final_dx_2'] 

other_feature = ['transudate','MPE', 'TPE_1', 'TPE_1_why', 'TPE_2', 'TPE_2_why', 'PPE', 'dx_other', 'dx_other_detail', 'excluded', 'ex_why', 'mannual_review']


print(len(info_feature),len(clinical_feature),len(blood_serum_feature),len(pleural_fluid_feature),len(target_features),len(date_time_feature),len(date_feature),len(other_feature))
print(len(info_feature)+len(clinical_feature)+len(blood_serum_feature)+len(pleural_fluid_feature)+len(target_features)+len(date_time_feature)+len(date_feature)+len(other_feature))

### Data Type Transfer
- 데이터 타입 변환
- 날짜 데이터는 일자시간타입으로 변환 
- 범주형 변수는 Object 타입으로 변환

In [ ]:
# Date 정보를 datetime64 로 변환
for date_c in date_feature:
    df[date_c] = pd.to_datetime(df[date_c])

In [ ]:
# Date Time정보를 datetime64 로 변환
for dt_c in date_time_feature:
    df[dt_c] = pd.to_datetime(df[dt_c])

In [ ]:
# 범주형 변수의 경우 Object 타입으로 변환 
df[categorical] = df[categorical].astype(object) 

### Date: Value -> Null
- According to Coding Book
    - BMI +- 14 days
    - BT +- 3 days
    - Blood/Serum +- 14 days

In [ ]:
null_df = pd.DataFrame(data=df.isnull().sum()/len(df),columns=['nan_ratio'])
null_df[null_df['nan_ratio'] > 0.8]

In [ ]:
# df['Drop'] = np.nan
bmi_ = 0 
bt_ = 0 
for idx in range(len(df)):
    if (abs((df.loc[idx, 'Pf_date'] - df.loc[idx,'Weight_date']).days) > 14):  # BMI & Body Temperature
        df.loc[idx, 'BMI'] = np.nan
        bmi_+=1
    if (abs((df.loc[idx, 'Pf_date'] - df.loc[idx,'BT_date']).days) > 3):
        df.loc[idx, 'BT'] = np.nan
        bt_+=1
        
print(f'Missing Ratio of BMI: {(bmi_/len(df))*100:.2f}')
print(f'Missing Ratio of BT: {(bt_/len(df))*100:.2f}')

In [ ]:
df['b_glucose_new'] = np.nan
cnt = 0 
for idx in range(len(df)):
    if abs((df.loc[idx, 'Pf_date'] - df.loc[idx,'b_glucose_date']).days) < 14:
        df.loc[idx, 'b_glucose_new'] = df.loc[idx,'b_glucose']
    else:
        if abs((df.loc[idx, 'Pf_date'] - df.loc[idx,'b_glucose_poct_date']).days) < 14:
            df.loc[idx, 'b_glucose_new'] = df.loc[idx,'b_glucose_poct']
        else: 
            # df.loc[idx, 'Drop'] = True
            cnt += 1
            
print(f'Missing Ratio of Glucose: {(cnt/len(df))*100:.2f}')

In [ ]:
date_list = ['b_WBC_date', 'b_Neutro_date', 'b_Lympho_date','b_mono_date','b_Eosinophil_date', 'b_Basophil_date', 'b_RBC_date','b_Hb_date', 
             'b_plt_date', 'b_ESR_date', 'b_CRP_date', 'b_Protein_date','b_ALP_date', 'b_TB_date', 'b_BUN_date', 'b_Calcium_date', 
             'b_Chloride_date', 'b_Cholesterol_date','b_total_CO2_date', 'b_Creatinine_date', 'b_GGT_date', 'b_AST_date','b_ALT_date', 
             'b_Phosphorus_date', 'b_K_date', 'b_Na_date','b_Uric_Acid_date', 'b_albumin_date', 'b_LDH_date', 'b_CEA_date']

In [ ]:
for c in date_list:
    cnt = 0 
    for idx in range(len(df)):
        if abs((df.loc[idx, 'Pf_date'] - df.loc[idx, c]).days) > 14:
            df.loc[idx, c[:-5]] = np.nan
            cnt += 1
    # print(f'Missing Ratio of {c}: {(cnt/len(df))*100:.2f}')

In [ ]:
null_df = pd.DataFrame(data=df.isnull().sum()/len(df),columns=['nan_ratio'])
null_df[null_df['nan_ratio'] > 0.8]

### Drop Columns

In [ ]:
for c in df.columns:
    if (c in other_feature): #  (c in info_feature) | 
        df.drop(columns=c, inplace=True)

In [ ]:
for c in df.columns:
    if df[c].isnull().sum()/len(df) > 0.8: # if Null Ratio > 80%, drop  
        df.drop(columns=c, inplace=True)
    if (c == 'Pf_date') | (c == 'Pf_date_time'):
        pass
    elif ('_date' in c) | ('Height' in c) | ('Weight' in c) | ('SONO' in c):
        df.drop(columns=c, inplace=True)

In [ ]:
df['b_glucose'] = df['b_glucose_new']
df.drop(columns=['b_glucose_new','b_glucose_poct'], axis=1, inplace=True)
df.drop(columns='final_dx_2', inplace=True)

In [ ]:
df.drop(columns = 'Pf_Lymph_PMN', inplace=True)

In [ ]:
df.shape

### Replace

In [ ]:
# # 자료형 변환 
# df = df.replace({'Gender':'M'}, {'Gender':0})
# df = df.replace({'Gender':'F'}, {'Gender':1})
# df['Gender'] = df['Gender'].astype(object)
# print(df['Gender'].value_counts())

In [ ]:
for idx in range(len(df)):
    if (df.loc[idx,'Pf_Color'] == 2) | (df.loc[idx,'Pf_Color'] == 3):
        df.loc[idx,'Pf_Color_bloody'] = int(1) # Bloody -> yes
    elif (df.loc[idx,'Pf_Color'] == 1) | (df.loc[idx,'Pf_Color'] == 4) | (df.loc[idx,'Pf_Color'] == 5) | (df.loc[idx,'Pf_Color'] == 6):
        df.loc[idx,'Pf_Color_bloody'] = int(0) # Bloody -> No
    # df['Pf_Color_bloody'] = df['Pf_Color_bloody']#.astype(int)

In [ ]:
df['Pf_Color'] = df['Pf_Color_bloody'].astype(object)
df.drop(columns='Pf_Color_bloody', inplace=True)
df.rename(columns ={'Pf_Color':'Pf_Color_bloody'}, inplace=True)

In [ ]:
# df.rename(columns ={'final_dx_1':'Target'}, inplace=True)

### Drop Excluded Data

In [ ]:
# 진단 1 에 대해서 불필요한 Feature 제거
df_1 = df[df['final_dx_1']!=0]
df_1.reset_index(inplace=True, drop=True)
df_1.shape

In [ ]:
df_1 = df_1.replace({'final_dx_1':1},{'final_dx_1':0})
df_1 = df_1.replace({'final_dx_1':2},{'final_dx_1':1})
df_1 = df_1.replace({'final_dx_1':3},{'final_dx_1':2})
df_1 = df_1.replace({'final_dx_1':4},{'final_dx_1':3})
df_1 = df_1.replace({'final_dx_1':5},{'final_dx_1':4})
df_1['final_dx_1'].value_counts()

### External Validation Set

In [ ]:
# Date Time에 대해서 시간 순으로 재 나열
new_df = df_1.sort_values(by=['Pf_date_time'], ascending=True)

In [ ]:
# 01 - '2018-01-01'을 기준으로 External Validation Set을 만듬 
internal_df = new_df[new_df['Pf_date'] < '2018-01-01'].reset_index(drop=True)
external_df = new_df[new_df['Pf_date'] >= '2018-01-01'].reset_index(drop=True)
print(f"Length of Internal:{len(internal_df)}\nLength of External:{len(external_df)}")

In [ ]:
# 각 질환별 갯수를 확인
internal_df['final_dx_1'].value_counts()

In [ ]:
external_df['final_dx_1'].value_counts()

In [ ]:
# 진단 1에 대해서 데이터를 정리한 테이블을 csv 파일로 저장
internal_df.to_csv('PF_AI_Internal.csv', index=False) 
external_df.to_csv('PF_AI_External.csv', index=False) 